# Genehancer 
- https://genome.ucsc.edu/cgi-bin/hgTables?db=hg38&hgta_group=regulation&hgta_track=geneHancer&hgta_table=geneHancerGenesDoubleElite&hgta_doSchema=describe+table+schema
- https://www.weizmann.ac.il/molgen/genehancer-genome-wide-integration-enhancers-and-target-genes-genecards

- http://www.enhanceratlas.org/linkv2.php#l

In [ ]:
import pandas as pd 

In [ ]:
cat -t delta_DNAme_table.txt | head 

In [ ]:
cat -t SE_02_1028_SE_hg38.bed | head 

In [ ]:
!bedtools intersect \
    -b SE_02_1028_SE_hg38.bed \
    -a delta_DNAme_table.txt -wb

In [ ]:
pd.read_csv('SE_02_1028_SE_hg38.bed',sep='\t').head()

pd.read_csv('delta_DNAme_table.txt',sep='\t').head()